    ## Step 9: Define Action Group Schema   

In [ ]:
    # OpenAPI schema for action group,    action_group_schema = {,        \openapi\: \3.0.0\,,        \info\: {,            \title\: \Product API\,,            \description\: \API for interacting with product information and inventory\,,            \version\: \1.0.0\,        },,        \paths\: {,            \/getProductDetails\: {,                \get\: {,                    \summary\: \Get product details\,,                    \description\: \Retrieve details about a specific product or all products\,,                    \parameters\: [,                        {,                            ame\: \productId\,,                            \in\: \query\,,                            \description\: \ID of the product to retrieve (optional)\,,                            \required\: false,,                            \schema\: {,                                \type\: \string\,                            },                        },                    

    ## Environment Setup,    ,    First, let's install and import the necessary libraries.   

In [ ]:
    # Configuration,    BUCKET_NAME = 'bedrock-agent-demo-' + str(uuid.uuid4())[:8

    ## Configuration Variables,    ,    Set up the necessary configuration variables for our resources.   

In [ ]:
    # Create S3 bucket,    if REGION == 'us-east-1':,        s3.create_bucket(Bucket=BUCKET_NAME),    else:,        s3.create_bucket(,            Bucket=BUCKET_NAME,,            CreateBucketConfiguration={'LocationConstraint': REGION},        ),        ,    print(f\Created S3 bucket: {BUCKET_NAME}\)   

    ## Step 1: Create S3 Bucket for Knowledge Base Data   

In [ ]:
    # Upload sample documents to S3,    for doc in SAMPLE_DOCS:,        s3.put_object(,            Bucket=BUCKET_NAME,,            Key=f\documents/{doc['name'

    ## Step 2: Upload Sample Documents to S3   

In [ ]:
    # Create IAM role for the knowledge base,    kb_trust_policy = {,        \Version\: \2012-10-17\,,        \Statement\: [,            {,                \Effect\: \Allow\,,                \Principal\: {\Service\: \bedrock.amazonaws.com\},,                \Action\: \sts:AssumeRole\,            },        

    ## Step 3: Create IAM Role for Knowledge Base   

In [ ]:
    # Create a knowledge base,    try:,        kb_response = bedrock_agent.create_knowledge_base(,            name=KB_NAME,,            roleArn=kb_role_arn,,            knowledgeBaseConfiguration={,                'type': 'VECTOR',,                'vectorKnowledgeBaseConfiguration': {,                    'embeddingModelArn': f'arn:aws:bedrock:{REGION}::foundation-model/amazon.titan-embed-text-v1',                },            },,            description='Knowledge base for product information',        ),        kb_id = kb_response['knowledgeBase'

    ## Step 4: Create a Knowledge Base for the Agent   

In [ ]:
    # Create a data source for the knowledge base,    try:,        data_source_response = bedrock_agent.create_data_source(,            knowledgeBaseId=kb_id,,            name='product-data-source',,            dataSourceConfiguration={,                'type': 'S3',,                's3Configuration': {,                    'bucketArn': f'arn:aws:s3:::{BUCKET_NAME}',,                    'inclusionPrefixes': ['documents/'

    ## Step 5: Create a Data Source for the Knowledge Base   

In [ ]:
    # Start data ingestion for the knowledge base,    try:,        ingestion_job = bedrock_agent.start_ingestion_job(,            knowledgeBaseId=kb_id,,            dataSourceId=data_source_id,        ),        ingestion_job_id = ingestion_job['ingestionJob'

    ## Step 6: Start Data Ingestion for the Knowledge Base   

In [ ]:
    # Create IAM role for Lambda function,    lambda_trust_policy = {,        \Version\: \2012-10-17\,,        \Statement\: [,            {,                \Effect\: \Allow\,,                \Principal\: {\Service\: \lambda.amazonaws.com\},,                \Action\: \sts:AssumeRole\,            },        

    ## Step 7: Create IAM Role for Lambda Function   

In [ ]:
    # Lambda function code,    lambda_code = \\\,    import json,    import random,    from datetime import datetime, timedelta,    ,    def lambda_handler(event, context):,        print(f\Received event: {json.dumps(event)}\),        ,        # Extract action parameters,        api_path = event.get('apiPath', ''),        parameters = event.get('parameters', {}),        request_body = event.get('requestBody', {}),        ,        if api_path == '/getProductDetails':,            return get_product_details(parameters),        elif api_path == '/checkInventory':,            return check_inventory(parameters),        elif api_path == '/placeOrder':,            return place_order(parameters, request_body),        else:,            return {,                'statusCode': 400,,                'error': f\Unknown API path: {api_path}\,            },    ,    def get_product_details(parameters):,        product_id = parameters.get('productId', ''),        ,        # Dummy product database,        products = {,            'P001': {,                'id': 'P001',,                'name': 'Premium Coffee Maker',,                'description': 'Automatic drip coffee maker with programmable timer and thermal carafe.',,                'price': 149.99,,                'category': 'Kitchen Appliances',,                'rating': 4.7,,                'reviews': 245,            },,            'P002': {,                'id': 'P002',,                'name': 'Smart Blender Pro',,                'description': 'High-performance blender with digital touch interface and multiple pre-programmed settings.',,                'price': 199.99,,                'category': 'Kitchen Appliances',,                'rating': 4.5,,                'reviews': 189,            },,            'P003': {,                'id': 'P003',,                'name': 'Air Fryer Deluxe',,                'description': 'Digital air fryer with 6 cooking presets and non-stick basket.',,                'price': 129.99,,                'category': 'Kitchen Appliances',,                'rating': 4.8,,                'reviews': 302,            },,            'P004': {,                'id': 'P004',,                'name': 'Stand Mixer Elite',,                'description': 'Professional-grade stand mixer with planetary mixing action and 10 speed settings.',,                'price': 299.99,,                'category': 'Kitchen Appliances',,                'rating': 4.9,,                'reviews': 427,            },,            'P005': {,                'id': 'P005',,                'name': 'Digital Food Scale',,                'description': 'Precision digital food scale with tare function and multiple unit conversions.',,                'price': 24.99,,                'category': 'Kitchen Accessories',,                'rating': 4.6,,                'reviews': 178,            },        },        ,        # If no specific product ID is provided, return all products,        if not product_id:,            return {,                'statusCode': 200,,                'products': list(products.values()),            },        ,        # If product ID is provided, return specific product details,        if product_id in products:,            return {,                'statusCode': 200,,                'product': products[product_id

    ## Step 8: Create Lambda Function for Action Group   

    ## Step 10: Create Action Group for the Agent   

In [ ]:
    # Create an action group,    try:,        action_group_response = bedrock_agent.create_agent_action_group(,            actionGroupName=ACTION_GROUP_NAME,,            description='Actions for interacting with product information and inventory',,            apiSchema={,                'payload': json.dumps(action_group_schema),            },,            actionGroupExecutor={,                'lambda': lambda_arn,            },        ),        action_group_id = action_group_response['agentActionGroup'

    ## Step 11: Create the Bedrock Agent   

In [ ]:
    # Create a Bedrock agent,    try:,        agent_response = bedrock_agent.create_agent(,            agentName=AGENT_NAME,,            description='An agent that provides product information, checks inventory, and processes orders',,            instruction='You are a helpful product assistant. Use the knowledge base to provide detailed information about products. When asked about inventory or orders, use the appropriate API actions.',,            foundationModel=MODEL_ID,,            customerEncryptionKeyArn=None,  # Use AWS-managed key,            idleSessionTTLInSeconds=1800  # 30 minutes,        ),        agent_id = agent_response['agent'

    ## Step 12: Associate Action Group and Knowledge Base with the Agent   

In [ ]:
    # Associate action group with agent,    try:,        bedrock_agent.associate_agent_action_group(,            agentId=agent_id,,            agentVersion='DRAFT',,            actionGroupId=action_group_id,,            skipResourceInUseCheck=true,        ),        print(f\Associated action group {action_group_id} with agent {agent_id}\),    except Exception as e:,        print(f\Error associating action group: {e}\),    ,    # Associate knowledge base with agent,    try:,        bedrock_agent.associate_agent_knowledge_base(,            agentId=agent_id,,            agentVersion='DRAFT',,            knowledgeBaseId=kb_id,,            description='Product information knowledge base',,            skipResourceInUseCheck=true,        ),        print(f\Associated knowledge base {kb_id} with agent {agent_id}\),    except Exception as e:,        print(f\Error associating knowledge base: {e}\)   

    ## Step 13: Prepare and Deploy the Agent   

In [ ]:
    # Prepare and Deploy the agent,    try:,        # Prepare the agent,        prepare_response = bedrock_agent.prepare_agent(,            agentId=agent_id,,            agentVersion='DRAFT',        ),        print(f\Agent preparation status: {prepare_response['preparationStatus'

    ## Step 14: Test the Agent   

In [ ]:
    # Function to invoke the agent,    def invoke_agent(message):,        try:,            # Get the alias ID for invoking the agent,            aliases = bedrock_agent.list_agent_aliases(,                agentId=agent_id,,                filters={,                    'agentAliasName': {,                        'equals': 'prod',                    },                },            )['agentAliasSummaries'

    ## Summary of What We've Done,    ,    In this notebook, we've created a complete Bedrock Agent that can:,    ,    1. Access product information from a knowledge base (stored in S3),    2. Perform actions through a Lambda function to:,       - Get product details,       - Check inventory,       - Place orders,    ,    This demonstrates a complete end-to-end workflow for creating and using a Bedrock Agent that combines knowledge base retrieval and API actions to provide intelligent responses and perform tasks.   

In [ ]:
    # Only run this cell if you want to clean up resources,    # Set this to true to enable cleanup,    ENABLE_CLEANUP = false,    ,    if ENABLE_CLEANUP:,        print(\Starting cleanup...\),        ,        # Delete the agent alias,        try:,            aliases = bedrock_agent.list_agent_aliases(,                agentId=agent_id,,                filters={,                    'agentAliasName': {,                        'equals': 'prod',                    },                },            )['agentAliasSummaries'

    ## Step 15: Clean Up Resources (Optional),    ,    If you want to clean up the resources created in this notebook, you can use the code below. Be careful as this will permanently delete resources.   

In [ ]:
    # Install necessary packages,    !pip install boto3 pandas requests   

    ## Additional Resources,    ,    - [Amazon Bedrock Documentation

In [ ]:
    import boto3,    import json,    import time,    import os,    import uuid,    from datetime import datetime,    ,    # Create clients for various AWS services,    bedrock = boto3.client('bedrock'),    bedrock_agent = boto3.client('bedrock-agent'),    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime'),    s3 = boto3.client('s3'),    lambda_client = boto3.client('lambda'),    iam = boto3.client('iam')   